In [ ]:
# starting refactor of MATLAB
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def gen_crbm_gauss_mike_min(CRBM, InitData, numframes, fr):
    numGibbs = 30
    numdims = InitData.shape[1]

    # Initialize visible and hidden layers
    visible = np.zeros((numframes, CRBM['numdims']))
    visible[:CRBM['order'], :] = InitData[fr:fr + CRBM['order'], :]

    hidden = np.ones((numframes, CRBM['numhid']))

    for tt in range(CRBM['order'], numframes):
        # Initialize with noise
        visible[tt, :] = visible[tt - 1, :] + 0.0001 * np.random.randn(numdims)

        # Compute dynamic biases
        bistar = np.zeros((numdims, 1))
        for hh in range(CRBM['order']):
            bistar += CRBM['model']['A'][:, :, hh] @ visible[tt - hh, :].reshape(-1, 1)

        bjstar = np.zeros((CRBM['numhid'], 1))
        for hh in range(CRBM['order']):
            bjstar += CRBM['model']['B'][:, :, hh] @ visible[tt - hh, :].reshape(-1, 1)

        # Gibbs sampling
        for _ in range(numGibbs):
            bottomup = CRBM['model']['w'] @ visible[tt, :].reshape(-1, 1)
            eta = (bottomup / CRBM['gsd']) + CRBM['model']['bj'] + bjstar
            hposteriors = sigmoid(eta)

            hidden[tt, :] = (hposteriors.T > np.random.rand(1, CRBM['numhid'])).astype(float)

            topdown = CRBM['gsd'] * (hidden[tt, :] @ CRBM['model']['w'])
            visible[tt, :] = topdown + CRBM['model']['bi'].T + bistar.T

        # Final mean-field approximation
        topdown = CRBM['gsd'] * (hposteriors.T @ CRBM['model']['w'])
        visible[tt, :] = topdown + CRBM['model']['bi'].T + bistar.T

    return {
        'visible': visible,
        'hidden': hidden
    }
